# Data clean-up

Here, we prepare our data before visualization. Based on the election districts map from [NYC Open Data](https://data.cityofnewyork.us/City-Government/Election-Districts/h2n3-98hq), what we need are the digits / number of the assembly districts + digits / number of the election districts to match.

In [1]:
# importing libraries
import pandas as pd

In [2]:
df = pd.read_csv("electoral_district_votes.csv")
len(df)

719

In [3]:
df.dtypes

electoral_district              object
reported                        object
Harris-Walz_Democratic           int64
Trump-Vance_Republican           int64
Trump-Vance_Conservative         int64
Harris-Walz_Working_Families     int64
write_in                         int64
assembly_district               object
dtype: object

### We need a column that shows both the Assembly District and Electoral District numbers 

This follows the format of our map and geocodes for viz, i.e., *AE*0*ED*.

In [4]:
df["ad_number"] = df["assembly_district"].str.extract(r"(\d+)").astype(int)
df["ed_number"] = df["electoral_district"].str.extract(r"(\d+)").astype(int)

# Create new column with the desired format
df["ed_map_for_viz"] = df["ad_number"].astype(str) + df["ed_number"].astype(str).str.zfill(3)
df.drop(columns=["ad_number", "ed_number"], inplace=True)
df

,electoral_district,reported,Harris-Walz_Democratic,Trump-Vance_Republican,Trump-Vance_Conservative,Harris-Walz_Working_Families,write_in,assembly_district,ed_map_for_viz
0,ED 1,99.00%,371,168,7,26,3,AD 77,77001
1,ED 2,99.00%,503,200,7,17,6,AD 77,77002
2,ED 3,99.00%,471,144,18,12,2,AD 77,77003
3,ED 4,83.33%,94,27,2,6,2,AD 77,77004
4,ED 5,99.00%,470,178,11,11,2,AD 77,77005
...,...,...,...,...,...,...,...,...,...
714,ED 55,99.00%,448,166,13,18,8,AD 87,87055
715,ED 56,99.00%,4,3,0,1,0,AD 87,87056
716,ED 57,99.00%,0,0,0,0,0,AD 87,87057
717,ED 58,99.00%,0,0,0,0,0,AD 87,87058


### We need a column that adds the votes per candidate

Currently, we're seeing Harris-Walz votes from Democratic and Working Families voters, and Trump-Vance votes from Republican and Conservative voters. What we want are the totals.

In [5]:
df["trump_votes"] = df["Trump-Vance_Republican"] + df["Trump-Vance_Conservative"]
df["harris_votes"] = df["Harris-Walz_Democratic"] + df["Harris-Walz_Working_Families"]
df

,electoral_district,reported,Harris-Walz_Democratic,Trump-Vance_Republican,Trump-Vance_Conservative,Harris-Walz_Working_Families,write_in,assembly_district,ed_map_for_viz,trump_votes,harris_votes
0,ED 1,99.00%,371,168,7,26,3,AD 77,77001,175,397
1,ED 2,99.00%,503,200,7,17,6,AD 77,77002,207,520
2,ED 3,99.00%,471,144,18,12,2,AD 77,77003,162,483
3,ED 4,83.33%,94,27,2,6,2,AD 77,77004,29,100
4,ED 5,99.00%,470,178,11,11,2,AD 77,77005,189,481
...,...,...,...,...,...,...,...,...,...,...,...
714,ED 55,99.00%,448,166,13,18,8,AD 87,87055,179,466
715,ED 56,99.00%,4,3,0,1,0,AD 87,87056,3,5
716,ED 57,99.00%,0,0,0,0,0,AD 87,87057,0,0
717,ED 58,99.00%,0,0,0,0,0,AD 87,87058,0,0


### Vote differential per election district

We will get the difference in Harris and Trump votes. In this case, negative values will mean more votes for Harris (Democrat), and positive values will indicate more votes for Trump (Republican). This will bring clarity to what values we will be using for our choropleth map. 

What we will see in our viz, then, will be shades of red and blue that shows margins of victory. **Lands don't vote; people do.**

In [6]:
df["votes_value_for_viz"] = df["trump_votes"] - df["harris_votes"]
df

,electoral_district,reported,Harris-Walz_Democratic,Trump-Vance_Republican,Trump-Vance_Conservative,Harris-Walz_Working_Families,write_in,assembly_district,ed_map_for_viz,trump_votes,harris_votes,votes_value_for_viz
0,ED 1,99.00%,371,168,7,26,3,AD 77,77001,175,397,-222
1,ED 2,99.00%,503,200,7,17,6,AD 77,77002,207,520,-313
2,ED 3,99.00%,471,144,18,12,2,AD 77,77003,162,483,-321
3,ED 4,83.33%,94,27,2,6,2,AD 77,77004,29,100,-71
4,ED 5,99.00%,470,178,11,11,2,AD 77,77005,189,481,-292
...,...,...,...,...,...,...,...,...,...,...,...,...
714,ED 55,99.00%,448,166,13,18,8,AD 87,87055,179,466,-287
715,ED 56,99.00%,4,3,0,1,0,AD 87,87056,3,5,-2
716,ED 57,99.00%,0,0,0,0,0,AD 87,87057,0,0,0
717,ED 58,99.00%,0,0,0,0,0,AD 87,87058,0,0,0


In [7]:
df.to_csv("electoral_district_for_viz.csv", encoding="UTF-8", index=False)